# **Import Library**

In [1]:
import os
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

# **Load Dataset**

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# Tentukan direktori tempat Anda menyimpan data CSV
# DATADIR = 'lokasi_folder_data_csv'
TRAIN_TEST_CUTOFF = '2016-04-21'
TRAIN_VALID_RATIO = 0.75

# Lokasi URL dataset di GitHub
dataset_urls = [
    'https://raw.githubusercontent.com/arifadli/DatasetRepository/main/FinansialPrediction/Processed_DJI.csv',
    'https://raw.githubusercontent.com/arifadli/DatasetRepository/main/FinansialPrediction/Processed_NASDAQ.csv',
    'https://raw.githubusercontent.com/arifadli/DatasetRepository/main/FinansialPrediction/Processed_NYSE.csv',
    'https://raw.githubusercontent.com/arifadli/DatasetRepository/main/FinansialPrediction/Processed_RUSSELL.csv',
    'https://raw.githubusercontent.com/arifadli/DatasetRepository/main/FinansialPrediction/Processed_SP.csv'
]

# Membaca dataset dari URL dan menyimpannya dalam kamus
data = {}
for url in dataset_urls:
    name = os.path.splitext(os.path.basename(url))[0]  # Ambil nama dataset dari URL
    X = pd.read_csv(url, index_col="Date", parse_dates=True)
    # Periksa apakah kolom "Name" ada dalam DataFrame X
    if "Name" in X.columns:
        del X["Name"]  # Hapus kolom "Name"
    # Pra-pemrosesan dasar: ambil nama, klasifikasi
    cols = X.columns
    X["Target"] = (X["Close"].pct_change().shift(-1) > 0).astype(int)
    X.dropna(inplace=True)
    # Sesuaikan scaler standar menggunakan dataset pelatihan
    index = X.index[X.index < TRAIN_TEST_CUTOFF]
    index = index[:int(len(index) * TRAIN_VALID_RATIO)]
    scaler = StandardScaler().fit(X.loc[index, cols])
    # Simpan dataframe yang telah diubah skala
    X[cols] = scaler.transform(X[cols])
    data[name] = X

# **Function Definition**

In [5]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1macro(y_true, y_pred):
    f_pos = f1_m(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = f1_m(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2

def cnnpred_2d(seq_len=60, n_features=82, n_filters=(8,8,8), droprate=0.1):
    "2D-CNNpred model according to the paper"
    model = Sequential([
        Input(shape=(seq_len, n_features, 1)),
        Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
        Conv2D(n_filters[1], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Conv2D(n_filters[2], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Flatten(),
        Dropout(droprate),
        Dense(1, activation="sigmoid")
    ])
    return model

def datagen(data, seq_len, batch_size, targetcol, kind):
    "As a generator to produce samples for Keras model"
    batch = []
    while True:
        # Pick one dataframe from the pool
        key = random.choice(list(data.keys()))
        df = data[key]
        input_cols = [c for c in df.columns if c != targetcol]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)
        assert split > seq_len, "Training data too small for sequence length {}".format(seq_len)
        if kind == 'train':
            index = index[:split]   # range for the training set
        elif kind == 'valid':
            index = index[split:]   # range for the validation set
        else:
            raise NotImplementedError
        # Pick one position, then clip a sequence length
        while True:
            t = random.choice(index)     # pick one time step
            n = (df.index == t).argmax() # find its position in the dataframe
            if n-seq_len+1 < 0:
                continue # this sample is not enough for one sequence length
            frame = df.iloc[n-seq_len+1:n+1]
            batch.append([frame[input_cols].values, df.loc[t, targetcol]])
            break
        # if we get enough for a batch, dispatch
        if len(batch) == batch_size:
            X, y = zip(*batch)
            X, y = np.expand_dims(np.array(X), 3), np.array(y)
            yield X, y
            batch = []

def testgen(data, seq_len, targetcol):
    "Return array of all test samples"
    batch = []
    for key, df in data.items():
        input_cols = [c for c in df.columns if c != targetcol]
        # find the start of test sample
        t = df.index[df.index >= TRAIN_TEST_CUTOFF][0]
        n = (df.index == t).argmax()
        # extract sample using a sliding window
        for i in range(n+1, len(df)+1):
            frame = df.iloc[i-seq_len:i]
            batch.append([frame[input_cols].values, frame[targetcol][-1]])
    X, y = zip(*batch)
    return np.expand_dims(np.array(X),3), np.array(y)

# **Building Model**

In [10]:
seq_len = 60
batch_size = 128
n_epochs = 20
n_features = 82

# Produce CNNpred as a binary classification problem
model = cnnpred_2d(seq_len, n_features)
model.compile(optimizer="adam", loss="mae", metrics=["acc", f1macro])
model.summary()  # print model structure to console

# Set up callbacks and fit the model
# We use custom validation score f1macro() and hence monitor for "val_f1macro"
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.h5"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max",
                    verbose=0, save_best_only=True, save_weights_only=False, save_freq="epoch")
]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 1, 8)          664       
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 1, 8)          200       
                                                                 
 max_pooling2d (MaxPooling2  (None, 29, 1, 8)          0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 1, 8)          200       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 1, 8)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 104)               0

# **Model Evaluation**

In [11]:
model.fit(datagen(data, seq_len, batch_size, "Target", "train"),
          validation_data=datagen(data, seq_len, batch_size, "Target", "valid"),
          epochs=n_epochs, steps_per_epoch=400, validation_steps=10, verbose=1, callbacks=callbacks)

# Prepare test data
test_data, test_target = testgen(data, seq_len, "Target")

# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.5).astype(int)
print("accuracy:", accuracy_score(test_pred, test_target))
print("MAE:", mean_absolute_error(test_pred, test_target))
print("F1:", f1_score(test_pred, test_target))

Epoch 1/20
400/400 [==============================] - 154s 372ms/step - loss: 0.4460 - acc: 0.5585 - f1macro: 0.3616 - val_loss: 0.4747 - val_acc: 0.5250 - val_f1macro: 0.3436
Epoch 2/20
  1/400 [..............................] - ETA: 4s - loss: 0.5107 - acc: 0.4844 - f1macro: 0.3263

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 149s 373ms/step - loss: 0.3549 - acc: 0.6618 - f1macro: 0.5939 - val_loss: 0.4559 - val_acc: 0.5539 - val_f1macro: 0.5111
Epoch 3/20
  1/400 [..............................] - ETA: 4s - loss: 0.2914 - acc: 0.7344 - f1macro: 0.7116

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 150s 375ms/step - loss: 0.2426 - acc: 0.7767 - f1macro: 0.7641 - val_loss: 0.4861 - val_acc: 0.5102 - val_f1macro: 0.4758
Epoch 4/20
400/400 [==============================] - 149s 372ms/step - loss: 0.2025 - acc: 0.8100 - f1macro: 0.8003 - val_loss: 0.5002 - val_acc: 0.5109 - val_f1macro: 0.4838
Epoch 5/20
400/400 [==============================] - 143s 360ms/step - loss: 0.1850 - acc: 0.8252 - f1macro: 0.8164 - val_loss: 0.4861 - val_acc: 0.5164 - val_f1macro: 0.4892
Epoch 6/20
400/400 [==============================] - 149s 374ms/step - loss: 0.1757 - acc: 0.8324 - f1macro: 0.8244 - val_loss: 0.4871 - val_acc: 0.5109 - val_f1macro: 0.4831
Epoch 7/20
400/400 [==============================] - 145s 364ms/step - loss: 0.1716 - acc: 0.8346 - f1macro: 0.8276 - val_loss: 0.4790 - val_acc: 0.5172 - val_f1macro: 0.4838
Epoch 8/20
400/400 [==============================] - 145s 363ms/step - loss: 0.1661 - acc: 0.8397 - f1macro: 0.8326 - val_loss: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 152s 381ms/step - loss: 0.1478 - acc: 0.8553 - f1macro: 0.8488 - val_loss: 0.4850 - val_acc: 0.5094 - val_f1macro: 0.4898
Epoch 16/20
400/400 [==============================] - 139s 348ms/step - loss: 0.1471 - acc: 0.8557 - f1macro: 0.8495 - val_loss: 0.4882 - val_acc: 0.5109 - val_f1macro: 0.4904
Epoch 17/20
400/400 [==============================] - 154s 385ms/step - loss: 0.1429 - acc: 0.8605 - f1macro: 0.8550 - val_loss: 0.4675 - val_acc: 0.5336 - val_f1macro: 0.5106
Epoch 18/20
400/400 [==============================] - 138s 347ms/step - loss: 0.1445 - acc: 0.8584 - f1macro: 0.8526 - val_loss: 0.4722 - val_acc: 0.5266 - val_f1macro: 0.5082
Epoch 19/20
400/400 [==============================] - 155s 388ms/step - loss: 0.1359 - acc: 0.8668 - f1macro: 0.8615 - val_loss: 0.4682 - val_acc: 0.5336 - val_f1macro: 0.5044
Epoch 20/20
33/33 [==============================] - 0s 3ms/step
accuracy: 0.5170731707317073
MAE: 0.48292682926829267
F1: 0.62